In [2]:
import pandas as pd
import re

def clean_tecnocasa_dataset(input_file):
    """
    Clean and transform the Tecnocasa dataset by removing unnecessary columns,
    extracting features, and standardizing data formats.
    """
    # Step 1: Load the dataset and remove unnecessary columns
    df = pd.read_csv(input_file)

    # Remove unnamed columns and image URL
    columns_to_drop = [col for col in df.columns if col.startswith('Unnamed:') or col == 'Image URL']
    df = df.drop(columns=columns_to_drop)

    # Remove subtitle column
    if 'Subtitle' in df.columns:
        df = df.drop(columns=['Subtitle'])

    # Step 2: Remove rows with 'Duplex' in the title
    df = df[~df['Title'].str.contains('Duplex', case=False, na=False)]

    # Step 3: Extract number of rooms from Title
    def extract_rooms(title):
        match = re.search(r'S\+(\d+)', title)
        return int(match.group(1)) if match else None

    df['Rooms'] = df['Title'].apply(extract_rooms)
    df = df.drop(columns=['Title'])

    # Step 4: Clean Surface column
    def clean_surface(surface):
        match = re.search(r'(\d+)\s*m²', str(surface))
        return int(match.group(1)) if match else None

    df['Surface'] = df['Surface'].apply(clean_surface)

    # Step 5: Extract region from URL
    def extract_region(url):
        match = re.search(r'grand-tunis/([^/]+)', str(url))
        if match:
            return match.group(1).replace('-', ' ')
        return None

    df['Region'] = df['URL'].apply(extract_region)
    df = df.drop(columns=['URL'])

    # Step 6: Extract boolean features
    def extract_boolean_features(features):
        features_str = str(features)

        ascenseur = 1 if 'Ascenseur: Oui' in features_str else 0
        chauffage_central = 1 if 'Chauffage: Centrale' in features_str else 0
        climatisation = 1 if 'Climatisation' in features_str else 0

        return pd.Series({
            'Ascenseur': ascenseur,
            'Chauffage_Central': chauffage_central,
            'Climatisation': climatisation
        })

    df[['Ascenseur', 'Chauffage_Central', 'Climatisation']] = df['Features'].apply(extract_boolean_features)

    # Step 7: Extract and clean Etage
    def clean_etage(features):
        features_str = str(features)

        # Look for specific words
        match = re.search(r'\b(dernier|moyen|terre)\b', features_str, re.IGNORECASE)
        if match:
            return match.group(0).lower()
        return None

    df['Etage'] = df['Features'].apply(clean_etage)

    # Step 8: Extract category (modified to capture only the first word)
    def extract_categorie(features):
        if pd.isna(features):
            return ""
        # Match only the first word after "Catégorie:"
        match = re.search(r"Catégorie:\s*(\w+)", str(features))
        if match:
            return match.group(1).strip()
        return ""

    df['Categorie'] = df['Features'].apply(extract_categorie)

    # Step 9: Remove remaining unnecessary columns
    columns_to_remove = ["Description", "Features", "Points of Interest", "Etage"]
    df = df.drop(columns=columns_to_remove)

    return df

# Example usage
if __name__ == "__main__":
    input_file = 'TechnocasaDataset.csv'
    output_file = 'cleaned_tecnocasa_dataset.csv'

    # Clean the dataset
    cleaned_df = clean_tecnocasa_dataset(input_file)

    # Save the cleaned dataset
    cleaned_df.to_csv(output_file, index=False)

    # Display information about the cleaned dataset
    print("\nCleaned Dataset Information:")
    print(cleaned_df.info())
    print("\nFirst few rows of the cleaned dataset:")
    print(cleaned_df.head())


Cleaned Dataset Information:
<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 1 to 221
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Price              185 non-null    int64 
 1   Surface            185 non-null    int64 
 2   Rooms              185 non-null    int64 
 3   Region             185 non-null    object
 4   Ascenseur          185 non-null    int64 
 5   Chauffage_Central  185 non-null    int64 
 6   Climatisation      185 non-null    int64 
 7   Categorie          185 non-null    object
dtypes: int64(6), object(2)
memory usage: 13.0+ KB
None

First few rows of the cleaned dataset:
     Price  Surface  Rooms               Region  Ascenseur  Chauffage_Central  \
1   317200       88      1               ariana          0                  0   
2  1500000      335      4    les berges du lac          1                  1   
3   280000      120      2            el menzah          0    